In [31]:
#Loading Data
import os
from pathlib import Path
import shutil

# Define paths
original_dataset_dir = "../Project4/images/Images"
base_dir = "../Project4/cnn-split"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Ensure directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define classes
classes = os.listdir(original_dataset_dir)

In [32]:
# Create directories for each class in train and test directories
for class_name in classes:
    class_train_dir = os.path.join(train_dir, class_name)
    class_test_dir = os.path.join(test_dir, class_name)
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_test_dir, exist_ok=True)

In [33]:
#Split data into train and test directories
split_ratio = 0.8  # 80% train, 20% test
for class_name in classes:
    class_images = os.listdir(os.path.join(original_dataset_dir, class_name)) #Gets labels based on dog subdirectories
    num_images = len(class_images) 
    num_train_images = int(split_ratio * num_images) #Calulates how many images total should be in the train directory (Not of each dog but total)

    # Copy images to train directory
    for image_name in class_images[:num_train_images]:
        src = os.path.join(original_dataset_dir, class_name, image_name)
        dst = os.path.join(train_dir, class_name, image_name)
        shutil.copyfile(src, dst)

    # Copy images to test directory
    for image_name in class_images[num_train_images:]:
        src = os.path.join(original_dataset_dir, class_name, image_name)
        dst = os.path.join(test_dir, class_name, image_name)
        shutil.copyfile(src, dst)

print("Data split and directories created successfully.")

Data split and directories created successfully.


In [34]:
#Check the splits worked
import os

# Define paths
base_dir = "../Project4/cnn-split"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Count the total number of images in train and test directories
#Using underscores to save memory by not storing labels,path, or directories and simply count number of files in each sub directory
total_train_count = sum(len(files) for _, _, files in os.walk(train_dir))
total_test_count = sum(len(files) for _, _, files in os.walk(test_dir))
total_images = total_test_count+total_train_count

print(f"Total Images: {total_images}")
print(f"Total Train Images: {total_train_count}, Percent of Total: {total_train_count/total_images}")
print(f"Total Test Images: {total_test_count}, Percent of Total: {total_test_count/total_images}")
print("Split is Effective!")


Total Images: 20580
Total Train Images: 16418, Percent of Total: 0.7977648202137998
Total Test Images: 4162, Percent of Total: 0.2022351797862002
Split is Effective!


In [74]:
#Rescaling of training data
#Train preprocessing
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
train_data_dir = '../Project4/cnn-split/train'

batch_size = 32
# target image size
img_height = 224
img_width = 224

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 16418 files belonging to 120 classes.
Using 13135 files for training.
Using 3283 files for validation.


In [75]:
#Rescaling of testing data
#Test preprocessing
test_data_dir = '../Project4/cnn-split/test/'

batch_size = 2 #using smaller batch size

# this is what was used in the paper --
img_height = 224
img_width = 224

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4162 files belonging to 120 classes.


In [70]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers, optimizers,
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [43]:
#Now on to training our Models!

In [91]:
#ResNets Model

ResNets = models.Sequential()

#Layer 1: Convolutional layer with 64 filters of size 7x3, followed by average pooling
ResNets.add(layers.Conv2D(64, kernel_size=(7, 7), activation='relu', input_shape=(224,224,3)))
ResNets.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

# Residual block 1
ResNets.add(layers.Conv2D(64, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))
ResNets.add(layers.Conv2D(64, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))

#Layer 2
ResNets.add(layers.Conv2D(128, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))
ResNets.add(layers.Conv2D(128, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))

#Layer 3
ResNets.add(layers.Conv2D(256, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))
ResNets.add(layers.Conv2D(256, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))

#Layer 4
ResNets.add(layers.Conv2D(512, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))
ResNets.add(layers.Conv2D(512, kernel_size=(3,3),strides=1, activation='relu',  padding="same"))
#Global average pooling 

ResNets.add(layers.GlobalAveragePooling2D())

# Flatten the feature maps to feed into fully connected layers
ResNets.add(layers.Flatten())


# Layer 3: Fully connected layer with 120 neurons
ResNets.add(layers.Dense(120, activation='softmax'))
#ResNets.add(layers.Dropout(.3)) #More moderate droprate for overfitting
# Compile model
ResNets.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
ResNets.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_196 (Conv2D)         (None, 218, 218, 64)      9472      
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 108, 108, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_197 (Conv2D)         (None, 108, 108, 64)      36928     
                                                                 
 conv2d_198 (Conv2D)         (None, 108, 108, 64)      36928     
                                                                 
 conv2d_199 (Conv2D)         (None, 108, 108, 128)     73856     
                                                                 
 conv2d_200 (Conv2D)         (None, 108, 108, 128)     147584    
                                                     

In [73]:
#fit the model from image generator
history = ResNets.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
411/411 [==============================] - 43s 103ms/step - loss: 7.9292 - accuracy: 0.0078 - val_loss: 4.7862 - val_accuracy: 0.0082
Epoch 2/20
411/411 [==============================] - 42s 103ms/step - loss: 7.9635 - accuracy: 0.0097 - val_loss: 4.7844 - val_accuracy: 0.0067
Epoch 3/20
411/411 [==============================] - 42s 102ms/step - loss: 7.9021 - accuracy: 0.0123 - val_loss: 4.7742 - val_accuracy: 0.0079
Epoch 4/20
411/411 [==============================] - 45s 109ms/step - loss: 7.8847 - accuracy: 0.0135 - val_loss: 4.7530 - val_accuracy: 0.0158
Epoch 5/20
411/411 [==============================] - 46s 111ms/step - loss: 7.8368 - accuracy: 0.0174 - val_loss: 4.7154 - val_accuracy: 0.0180
Epoch 6/20
411/411 [==============================] - 45s 109ms/step - loss: 7.8140 - accuracy: 0.0179 - val_loss: 4.6826 - val_accuracy: 0.0213
Epoch 7/20
411/411 [==============================] - 45s 110ms/step - loss: 7.8716 - accuracy: 0.0198 - val_loss: 4.6497 - val_ac

KeyboardInterrupt: 

In [ ]:
#Model VGGNet 16
VGGNet = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
VGGNet.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))
VGGNet.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
VGGNet.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
VGGNet.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
VGGNet.add(layers.Flatten())


# Layer 3: Fully connected layer with 120 neurons
VGGNet.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
VGGNet.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
VGGNet.add(layers.Dense(2, activation='softmax'))

# Compile model
VGGNet.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
VGGNet.summary()

In [ ]:
#fit the model from image generator
history = VGGNet.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

In [76]:
#Model AlexNet
AlexNet = models.Sequential()

# Layer 1: Convolutional layer with 96 filters of size 11x11, followed by max pooling
AlexNet.add(layers.Conv2D(96, kernel_size=(11,11),strides=4, activation='relu', input_shape=(224, 224, 3)))
AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

AlexNet.add(layers.Conv2D(256, kernel_size=(5,5),padding='same', activation='relu'))
AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

AlexNet.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet.add(layers.Conv2D(384, kernel_size=(3,3),padding='same', activation='relu'))
AlexNet.add(layers.Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))

AlexNet.add(layers.MaxPooling2D(pool_size=(3, 3),strides=2))

# Flatten the feature maps to feed into fully connected layers
AlexNet.add(layers.Flatten())


# Layer 3: Fully connected layer with 120 neurons
AlexNet.add(layers.Dense(4096, activation='relu'))
AlexNet.add(layers.Dropout(.5))
# Layer 4: Fully connected layer with 84 neurons
AlexNet.add(layers.Dense(4096, activation='relu'))
AlexNet.add(layers.Dropout(.5))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
AlexNet.add(layers.Dense(120, activation='softmax'))

# Compile model
AlexNet.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
AlexNet.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_83 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 26, 26, 96)        0         
 g2D)                                                            
                                                                 
 conv2d_84 (Conv2D)          (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 12, 12, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_85 (Conv2D)          (None, 12, 12, 384)       885120    
                                                                 
 conv2d_86 (Conv2D)          (None, 12, 12, 384)     

In [77]:
#fit the model from image generator
history = AlexNet.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
101/411 [======>.......................] - ETA: 3:10 - loss: 4.7879 - accuracy: 0.0077

KeyboardInterrupt: 